In [139]:
## -- Trova il primo swing

def find_first_swing(signal):
    found = False
    swing_onset = 0
    for i in range(0,signal.shape[0]): 
        if not found:
            swing_onset += 1
        if signal[i,-1] == 1:
            found = True
    return swing_onset

In [138]:
## -- Elimina i falsi stance in mezzo agli swing - i falsi 0 in mezzo agli 1

def delete_false_stance(signal, thr):
    # prima devo trovare le transizioni - 1 -> 0
    transitions = []
    for i in range(0,signal.shape[0] - 1):
        if signal[i,-1] == 1 and signal[i+1,-1] == 0:
            transitions.append(i)

    # Qua congiungo i falsi stance in mezzo agli swing
    for k in transitions:
        if any(signal[k : k + thr, -1]) == 1:
            signal[k - thr : k + thr, -1] = 1
    return signal

In [137]:
## -- Elimina i falsi swing (non fisiologici) in base alla threshold che gli si da

# Devo trovare le altre transizioni - stance to swing - 0 -> 1
def delete_false_swing(signal, thr):
    transitions = []
    for i in range(0,signal.shape[0] - 1):
        if signal[i,-1] == 0 and signal[i+1, -1] == 1:
            transitions.append(j)

    # Adesso posso controllare se sono fisiologiche oppure no
    for k in transitions:
        if any(signal[k : k + thr, -1]) == 0:
            signal[k : k + thr, -1] = 0
    return signal

In [152]:
## -- Clean Basograhic Signal
# Questo script mi serve per eliminare tutti i "passi falsi" :-)

import numpy as np
from numpy import genfromtxt as csv_load
# import matplotlib.pyplot as plt
import pandas as pd

thr = 200

tipo = np.dtype(np.float32)    # Per farglielo caricare 

in_directory = '../subjects/'
prefix = 'Biphase_Subject'
suffix = '_norm.csv'
out_directory = '../subjects/clean_baso/'
out_path = out_directory + 'Real/'
out_prefix = 's'
baso_suffix = '_baso_real.csv'
subjects = 26


for subject in range(1, subjects + 1):
    # Carica il segnale (è un NUMPY)
    signal = csv_load(in_directory + prefix + str(subject) + suffix, dtype = tipo, delimiter = ',')

    # Calcola il primo swing e aggiorna il vettore
    lower_bound = find_first_swing(signal)
    signal = signal[lower_bound:,:]
    
    # Cancella le false transizioni 
    signal = delete_false_stance(signal, thr)
    signal = delete_false_swing(signal, thr)
    baso = signal[:,-1]
    
    signal = pd.DataFrame(signal)
    baso = pd.DataFrame(baso)
    signal = signal.to_csv(out_directory + out_prefix + str(subject) + suffix, index = None, header = None)    # Salvo il file completo
    baso = baso.to_csv(out_path + out_prefix + str(subject) + baso_suffix, index = None, header = None)    # Salvo solo il basografico